### Запуск QtLab

In [2]:
%run D:\qtlab_replacement\init.py
%matplotlib qt5
import setup_logging
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *
from matplotlib import interactive
interactive(True)
import os
import matplotlib.patches as plm

Failed loading module Signal_Hound_SA:  [WinError 126] The specified module could not be found


# Спектроскопия

In [3]:
pna = Agilent_N5242A('pna', address = 'PNA')
lo1 = Agilent_E8257D('lo1', address = 'GEN1')

In [4]:
current_src1 = Yokogawa_GS210(address = 'GPIB0::1::INSTR')
#current_src2 = Yokogawa_GS210(address = 'GPIB0::2::INSTR')

In [5]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")
#pna.do_set_sweep_mode("Log")

True

In [6]:
# AWG для того чтобы открыть смесители
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
Vdc_q = 0.2
Vdc_r = 0.2
# Задаём постояннmtые смещения на каналы 1 и 3
lo1.set_power(0)
pna.set_power(0)
awg_tek.clear_waveforms()
awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=1)
awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=2)
awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=3)
awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=4)
awg_tek.set_offset(Vdc_q, channel=1)
awg_tek.set_offset(Vdc_q, channel=2)
awg_tek.set_offset(Vdc_r, channel=3)
awg_tek.set_offset(Vdc_r, channel=4)
awg_tek.set_output(1, channel=1)
awg_tek.set_output(1, channel=2)
awg_tek.set_output(1, channel=3)
awg_tek.set_output(0, channel=4)

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


True

## Спектроскопия парампа

In [ ]:
current_src1.set_status(0)
current_src2.set_status(1)
lo1.set_status(0)
#lo2.set_status(0)
#pna.set_power(-50)
pna.set_nop(401)
pna.set_bandwidth(500)
current_nop=101
currents=np.linspace(-2e-3,2e-3,current_nop)
pna.set_xlim(6e9,10e9)
result = sweep.sweep(pna, (currents, current_src2.set_current, 'Coil current'), filename='paramp spectrum direct')

## Резонаторная спектроскопия

### Одномерный скан при фиксированном токе 

6.745,6.61,6,472 тест

In [137]:
qubit_params = {1: {'Fr': 6.7460e9,
                    'F01_max': 8.1755e9, 
                    'Iw':750e-9,
                    'paramp_bias':0.165e-3, 
                    'paramp_pump_freq':13.146e9, 
                    'paramp_pump_power':13.76},
                2: {'Fr': 6475700000,  
                    'F01_max': 8.0392e9,   
                    'Iw':500e-9,
                    'paramp_bias':0.1657e-3, 
                    'paramp_pump_freq':13.146e9, 
                    'paramp_pump_power':13.76},
                3: {'Fr': 6611200000,
                    'F01_max': 8.008e9, 
                    'Iw':100e-9, 
                    'paramp_bias':0.177e-3, 
                    'paramp_pump_freq':13.14e9, 
                    'paramp_pump_power':12.84},
                }
qubit_id=1

## CHG

In [51]:
qubit_params = {1: {'Fr': 6.657e9,
                    'F01_max': 8.1755e9, 
                    'Iw':750e-9,
                    'paramp_bias':0.165e-3, 
                    'paramp_pump_freq':13.146e9, 
                    'paramp_pump_power':13.76},
                2: {'Fr': 6.797e9,  
                    'F01_max': 8.0392e9,   
                    'Iw':500e-9,
                    'paramp_bias':0.1657e-3, 
                    'paramp_pump_freq':13.146e9, 
                    'paramp_pump_power':13.76},
                3: {'Fr': 6.809e9,
                    'F01_max': 8.008e9, 
                    'Iw':100e-9, 
                    'paramp_bias':0.177e-3, 
                    'paramp_pump_freq':13.14e9, 
                    'paramp_pump_power':12.84},
                }
qubit_id=1

## Кубитная спектроскопия

## Single Tone spectroscopy

Завимисость резонаторов от потока

In [191]:
current_src1.set_status(1)
lo1.set_status(0)
qubit_id=3
#lo2.set_status(0)
#pna.set_power(-35)
pna.set_nop(101)
#pna.set_bandwidth(200)
current_max=1e-3
current_nop=101
pna.set_power(-55)
pna.set_bandwidth(50)
pna.set_average(0)
pna.set_averages(1)
dfr=2e6
pna.set_xlim(qubit_params[qubit_id]['Fr']-dfr, qubit_params[qubit_id]['Fr']+dfr)
currents=np.linspace(-current_max,current_max,current_nop)
# pna.set_xlim(6.75e9,7.25e9)
result = sweep.sweep(pna, (currents, current_src1.set_current, 'Coil current'), filename='anticrossings Baumanka Vanya-3 #{}'.format(qubit_id))

A Jupyter Widget

Started at:  May 01 2018 23:00:42

Elapsed time: 0 h 3 m 29.45 s


In [10]:
import matplotlib.pyplot as plt
pna.set_power(-20)
pna.set_bandwidth(10)
pna.set_nop(1001)
pna.set_xlim(6.4e9,7.0e9)
fr=pna.get_freqpoints()
s21_amp= pna.get_tracedata()[0]
plt.plot(fr,s21_amp)

Одномерная однотонка

In [130]:
#current_src1.set_status(1)
#current_src1.set_current(0)
#lo1.set_status(0)
pna.set_nop(1001)
pna.set_power(-20)
pna.set_bandwidth(100)
pna.set_average(0)
pna.set_averages(1)
dfr=50e6
qubit_id=3 
pna.set_xlim(qubit_params[qubit_id]['Fr']-dfr,  qubit_params[qubit_id]['Fr']+dfr)
#pna.set_xlim(6.4e9, 6.8e9)
result = sweep.sweep(pna, ([0], lambda x: None, 'None'), filename='Single Tone spectroscopy #2 Baumanka Vanya-3')

A Jupyter Widget

Started at:  Apr 30 2018 19:18:36

Elapsed time: 0 h 0 m 9.36 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


Plot Delay

In [131]:
freqs=result["S-parameter"][1][0]
S21=result["S-parameter"][2]
diff=np.diff(np.unwrap(np.angle(S21)))
freqs_diff = (freqs[1:]+freqs[:-1])/2
plt.plot(freqs_diff,np.abs(diff))
plt.show()
center=freqs[np.argmax(np.abs(diff))]
center

6611200000.0

In [204]:
def curr(current):
    lo1.set_status(0)
    pna.set_average(0)
    pna.set_nop(1001)
    dfr=50e6
    pna.set_xlim(qubit_params[qubit_id]['Fr']-dfr,  qubit_params[qubit_id]['Fr']+dfr)
    pna.set_bandwidth(10)
    current_src1.set_current(current)
    S21   = pna.get_tracedata()
    freqs = pna.get_freqpoints()
    diff=np.diff(np.unwrap(S21[1]))
    freqs_diff = (freqs[1:]+freqs[:-1])/2
    pna.set_bandwidth(resolution_bw)
    pna.set_nop(1)
    center=freqs_diff[np.argmax(np.abs(diff))]
    pna.set_xlim(center,center)#freqs_diff[np.argmin(S21[0])]-0.2e6,freqs[np.argmin(S21[0])]-0.2e6)
    lo1.set_status(1)
    plt.plot(freqs_diff, np.abs(diff))

Unwrap phase

In [ ]:
S21=result['S-parameter'][2]
freqs = result['S-parameter'][1][1]
curr= result['S-parameter'][1][0]
#f=np.arctan(np.real(S21)/np.imag(S21))
f = np.angle(S21)
f_u = np.unwrap(f)
f_u_span = np.median(f_u[:,-1]-f_u[:,0])+2*np.pi*3
S21_c = S21*np.exp(-1j*np.linspace(0, f_u_span, len(f[0,:])))
plt.figure()
plt.pcolormesh(curr,freqs,np.angle(S21_c).T);
plt.colorbar()

## Two-tone 

In [206]:
pna.set_power (-60)
curr(0)

In [210]:
#### НЕЖМИ МИНЯ
current_src1.set_status(1)
#current_src1.set_current(1e-3)
#Измерение
qubit_id = 2
#curr(6e-5)
lo1.set_status(1)
ex_freqs = np.arange(5.4e9,6.4e9, 2e6)
#pna.set_xlim(qubit_params[qubit_id]['Fr'],qubit_params[qubit_id]['Fr'])
pna.set_nop(1)
pna.set_xlim(qubit_params[qubit_id]['Fr'], qubit_params[qubit_id]['Fr'])
pna.set_power (-60)
curr(-3e-3)
pna.set_bandwidth(1)
#pna.set_average(1)
pna.set_averages(1)
#pna.set_average_mode('POINT')
pna.set_average(0)
#pna.set_averages(1)
#pna.set_power (-40)
lo1.set_power (-20)
#current_src1.set_current(6e-5)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'), filename = "ex_{0:.1f}dBm cur{1}#{2}".format(lo1.get_power(),current_src1.get_current(),\
                                                                                                                                  qubit_id))
except KeyboardInterrupt: pass 

A Jupyter Widget

Started at:  May 02 2018 10:17:41


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)



Elapsed time: 0 h 9 m 33.05 s


In [52]:
current_src1.set_current(-4e-5)

True

In [50]:
lo1.set_status(0)
pna.set_nop(201)
#pna.set_centerfreq(6.656e9)
pna.set_centerfreq(6.799e9)
pna.set_bandwidth(200)
pna.set_span(200e6)
sweep.sweep(pna, ([1], lambda x: 0, ''))
#S21 = pna.measure()['S-parameter']
#plt.figure()
#plt.plot(pna.get_freqpoints(), np.abs(S21))


A Jupyter Widget

Started at:  Apr 27 2018 20:05:59


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)



Elapsed time: 0 h 0 m 1.14 s


{'S-parameter': (('Frequency',),
  (array([  6.69900000e+09,   6.70000000e+09,   6.70100000e+09,
            6.70200000e+09,   6.70300000e+09,   6.70400000e+09,
            6.70500000e+09,   6.70600000e+09,   6.70700000e+09,
            6.70800000e+09,   6.70900000e+09,   6.71000000e+09,
            6.71100000e+09,   6.71200000e+09,   6.71300000e+09,
            6.71400000e+09,   6.71500000e+09,   6.71600000e+09,
            6.71700000e+09,   6.71800000e+09,   6.71900000e+09,
            6.72000000e+09,   6.72100000e+09,   6.72200000e+09,
            6.72300000e+09,   6.72400000e+09,   6.72500000e+09,
            6.72600000e+09,   6.72700000e+09,   6.72800000e+09,
            6.72900000e+09,   6.73000000e+09,   6.73100000e+09,
            6.73200000e+09,   6.73300000e+09,   6.73400000e+09,
            6.73500000e+09,   6.73600000e+09,   6.73700000e+09,
            6.73800000e+09,   6.73900000e+09,   6.74000000e+09,
            6.74100000e+09,   6.74200000e+09,   6.74300000e+09,
       

In [196]:
qubit_params[2]['Fr']

6475700000

In [154]:
curr(-1e-3)

-0.001 6745950000.0


In [215]:
current_src1.set_status(1)
#lo1.set_status(1)
lo1.set_power(5)
pna.set_power(-60)
currents =np.linspace(-6e-3,6e-3,27)
ex_freqs =np.linspace(3.0e9,4.0e9,101)

def curr(current):
    lo1.set_status(0)
    current_src1.set_current(current)
    pna.set_nop(101)
    pna.set_bandwidth(10)
    pna.set_average(0)
    pna.set_averages(1)
    dfr=10e6
    pna.set_xlim(qubit_params[qubit_id]['Fr']-dfr,  qubit_params[qubit_id]['Fr']+dfr)
    S21   = pna.get_tracedata()
    freqs = pna.get_freqpoints()
    diff=np.diff(np.unwrap(S21[1]))
    freqs_diff = (freqs[1:]+freqs[:-1])/2
    pna.set_bandwidth(3)
    pna.set_nop(1)
    center=freqs_diff[np.argmax(np.abs(diff))]
    print(current,center)
    pna.set_xlim(center,center)#freqs_diff[np.argmin(S21[0])]-0.2e6,freqs[np.argmin(S21[0])]-0.2e6)
    lo1.set_status(1)

for qubit_id in [1,2,3]:
    results=sweep.sweep(pna,(currents,curr,'Current'),
                (ex_freqs,lo1.set_frequency,'Second Tone Frequency'),
                filename = "Baumanka Two-tone current-frequency #{}".format(qubit_id)) #"2-Tone Power{0} #{1}Baumanka".format(lo1.get_power(),qubit_id))
#sweep.sweep(pna,(currents,curr,'Current'),(ex_freqs,lo1.set_frequency,'Tone-2 Fr'),filename = "2-Tone Power{0} #{1}CHG".format(lo1.get_power(),qubit_id))

A Jupyter Widget

Started at:  May 03 2018 09:31:19
-0.006 6744500000.0
-0.00553846153846 6744700000.0
-0.00507692307692 6744900000.0
-0.00461538461538 6745100000.0
-0.00415384615385 6745100000.0
-0.00369230769231 6745100000.0
-0.00323076923077 6745100000.0
-0.00276923076923 6744900000.0
-0.00230769230769 6744900000.0
-0.00184615384615 6744900000.0
-0.00138461538462 6745100000.0
-0.000923076923077 6744700000.0
-0.000461538461538 6745900000.0
0.0 6746100000.0
0.000461538461538 6745900000.0
0.000923076923077 6745900000.0
0.00138461538462 6745700000.0
0.00184615384615 6745700000.0
0.00230769230769 6745700000.0
0.00276923076923 6745100000.0
0.00323076923077 6744900000.0
0.00369230769231 6744700000.0
0.00415384615385 6744700000.0
0.00461538461538 6744900000.0
0.00507692307692 6744500000.0
0.00553846153846 6743100000.0
0.006 6742900000.0

Elapsed time: 0 h 27 m 17.56 s


A Jupyter Widget

Started at:  May 03 2018 09:58:38
-0.006 6471800000.0
-0.00553846153846 6472200000.0
-0.00507692307692 6472400000.0
-0.00461538461538 6472400000.0
-0.00415384615385 6472600000.0
-0.00369230769231 6472600000.0
-0.00323076923077 6474800000.0
-0.00276923076923 6474800000.0
-0.00230769230769 6474800000.0
-0.00184615384615 6475000000.0
-0.00138461538462 6474800000.0
-0.000923076923077 6474600000.0
-0.000461538461538 6475200000.0
0.0 6474800000.0
0.000461538461538 6475400000.0
0.000923076923077 6475400000.0
0.00138461538462 6475200000.0
0.00184615384615 6475000000.0
0.00230769230769 6474600000.0
0.00276923076923 6474200000.0
0.00323076923077 6474000000.0
0.00369230769231 6473600000.0
0.00415384615385 6473400000.0
0.00461538461538 6473000000.0
0.00507692307692 6472800000.0
0.00553846153846 6471400000.0
0.006 6471200000.0

Elapsed time: 0 h 27 m 44.23 s


A Jupyter Widget

Started at:  May 03 2018 10:26:23
-0.006 6608300000.0
-0.00553846153846 6608500000.0
-0.00507692307692 6608700000.0
-0.00461538461538 6608700000.0
-0.00415384615385 6608900000.0
-0.00369230769231 6608700000.0
-0.00323076923077 6608700000.0
-0.00276923076923 6608300000.0
-0.00230769230769 6610900000.0
-0.00184615384615 6610700000.0
-0.00138461538462 6610700000.0
-0.000923076923077 6610700000.0
-0.000461538461538 6610500000.0
0.0 6611100000.0
0.000461538461538 6610900000.0
0.000923076923077 6611100000.0
0.00138461538462 6611100000.0
0.00184615384615 6610900000.0
0.00230769230769 6610700000.0
0.00276923076923 6610100000.0
0.00323076923077 6609900000.0
0.00369230769231 6609700000.0
0.00415384615385 6609900000.0
0.00461538461538 6609700000.0
0.00507692307692 6607900000.0
0.00553846153846 6607700000.0
0.006 6607500000.0

Elapsed time: 0 h 27 m 35.72 s


In [101]:
S21  = pna.get_tracedata()
freqs = pna.get_freqpoints()
diff=np.diff(np.unwrap(S21[1]))
freqs_diff = (freqs[1:]+freqs[:-1])/2
center=freqs_diff[np.argmax(np.abs(diff))]
print(center)

ValueError: attempt to get argmax of an empty sequence

In [92]:
S21   = pna.get_tracedata()

In [94]:
np.unS21[1]

array([ 0.15564658])

In [18]:
qubit_id=1

In [35]:
import pickle as pkl

In [36]:
with open(r'D:\qtlab\ReiData\data\2018-04-27\12-55-50-2-Tone Power15.0 #3CHG\S-parameter 2-Tone Power15.0 #3CHG.pkl','rb') as f:
            data_r=pkl.load(f,encoding='latin1')

In [41]:
data_r[1]['S-parameter'][2]

array([[-0.00559385-0.00392395j, -0.00597066-0.00211769j,
        -0.00599210-0.00208535j, ..., -0.00599881-0.00193179j,
        -0.00595433-0.00197213j, -0.00599618-0.00189985j],
       [-0.00567988-0.00226442j, -0.00591274-0.00299845j,
        -0.00591286-0.00308009j, ..., -0.00594207-0.00279516j,
        -0.00590490-0.0027569j , -0.00593390-0.00283849j],
       [-0.00582377-0.00409298j, -0.00582671-0.00416346j,
        -0.00589933-0.0041504j , ..., -0.00590423-0.00389612j,
        -0.00593536-0.00389928j, -0.00592194-0.00393025j],
       ..., 
       [-0.00561414-0.00436587j, -0.00568742-0.00442881j,
        -0.00566091-0.0044215j , ..., -0.00574829-0.00416149j,
        -0.00571793-0.00421532j, -0.00574697-0.00420305j],
       [-0.00552052-0.00507544j, -0.00548747-0.00511744j,
        -0.00544493-0.00509442j, ..., -0.00559078-0.00493088j,
        -0.00559082-0.00484655j, -0.00560170-0.00483351j],
       [-0.00525024-0.0055186j , -0.00517460-0.00553628j,
        -0.00526264-0.0055827

In [42]:
S21=data_r[1]['S-parameter'][2]
freqs = data_r[1]['S-parameter'][1][1]
curr= data_r[1]['S-parameter'][1][0]
#f=np.arctan(np.real(S21)/np.imag(S21))
f = np.angle(S21)
f_u = np.unwrap(f)
f_u_span = np.median(f_u[:,-1]-f_u[:,0])+2*np.pi*3
S21_c = S21*np.exp(-1j*np.linspace(0, f_u_span, len(f[0,:])))
plt.figure()
plt.pcolormesh(curr,freqs,np.angle(S21_c).T);
plt.colorbar()

In [43]:
qubit_id

3

In [195]:
current_src1.set_status(1)
lo1.set_status(1)
#power = -20
#lo1.set_power(power)
pna.set_bandwidth(1)
pna.set_power(-66)
pna.set_nop(1)
pna.set_average(0)
pna.set_averages(1)
#curr(122e-6)
current_src1.set_current(0)
ex_freqs =np.linspace(5.6e9, 6.6e9, 201)
ex_powers = np.linspace(-20, 0, 21)
for qubit_id in [1,2,3]:
    pna.set_xlim(qubit_params[qubit_id]['Fr'],  qubit_params[qubit_id]['Fr'])
    sweep.sweep(pna, 
            (ex_powers, lo1.set_power, '2nd tone Power'), 
            (ex_freqs, lo1.set_frequency, '2nd tone frequency'), 
            filename='Two-tone 2D qubit {} power scan'.format(qubit_id))

A Jupyter Widget

Started at:  May 01 2018 23:34:01

Elapsed time: 1 h 18 m 44.81 s


A Jupyter Widget

Started at:  May 02 2018 00:52:47

Elapsed time: 1 h 19 m 24.07 s


A Jupyter Widget

Started at:  May 02 2018 02:12:13

Elapsed time: 1 h 19 m 37.05 s


In [ ]:
current_src1.set_status(1)
lo1.set_status(1)
#power = -20
#lo1.set_power(power)
pna.set_bandwidth(1)
pna.set_power(-65)
pna.set_nop(1)
#curr(122e-6)
current_src1.set_current(0)
ex_freqs 
ex_freqs =+np.linspace(-400e6, 100e6, 501)
pna.set_frequency(qubit_params)
sweep.sweep(pna, (ex_freqs, lo1.set_frequency, '2nd tone frequency'), filename='Two-tone 1D qubit {} power {}'.format(qubit_id, int(power)))

# Импульсы

### Создаём девайсы

In [7]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = lo1
lo_ro = pna
#lo_ro = Labbrick(name='lo_ro', serial=15257)
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


### Загружаем импульсные скрипты всякие

In [8]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)

<module 'sweep' from 'D:\\qtlab_replacement\\scripts\\sweep.py'>

### Осцилляторы

In [9]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
#lo_ex_pow = 14
lo_ex_pow = 14
#lo_ex = lo1
lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
#lo_ro_pow = 16
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

#lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

True

### AWGшки

In [10]:
# промежуточные частоты для гетеродинной схемы:
ex_if = 250e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 0.5e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 2В.
for channel in range(1,5):
    awg_tek.set_amplitude(.2, channel=channel)
    awg_tek.set_offset(0, channel=channel)
    awg_tek.set_output(1, channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 2, 1, lo_ex)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(4.8e9)
iq_ro.set_frequency(6.8e9)

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-0.0, channel=1)
awg_tek.set_marker2_low(-0.0, channel=1)
awg_tek.set_marker1_high(1.0, channel=1)
awg_tek.set_marker2_high(1.0, channel=1)
awg_tek.set_marker1_low(-0.04, channel=2)
awg_tek.set_marker2_low(-0.04, channel=2) 
awg_tek.set_marker1_high(0.04, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.04, channel=2) # с клока хватит и такой амплитуды
ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)
awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
awg_tek.run()

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


### Оцифровщик

In [13]:
# настройки оцифровщика
adc.set_clock(ro_clock)
adc.reset()            
adc.set_timeout(10000)


#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(400) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(600) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())

### Калибровка миксеров

In [11]:
#sa.set_ref(0)
iq_ex.get_calibration(sa)
iq_ro.get_calibration(sa)

d:\qtlab\ReiData\data/calibrations//IQ-if2.5e+08-rf4.8e+09-sb--1.pkl
5050000000.0
[ 0.3  0.3] -30.5720176697
[ 0.315  0.3  ] -30.3380222321
[ 0.3    0.315] -30.2316474915
[ 0.315  0.285] -30.6081962585
[ 0.3225  0.27  ] -30.800403595
[ 0.3075  0.27  ] -31.0404262543
[ 0.30375  0.255  ] -31.4691791534
[ 0.32625  0.225  ] -31.6222782135
[ 0.339375  0.1875  ] -32.0853157043
[ 0.320625  0.1725  ] -32.8400039673
[ 0.3196875  0.12375  ] -33.8315696716
[ 0.3553125  0.05625  ] -33.6105194092
[ 0.335625 -0.0075  ] -34.4063072205
[ 0.33375 -0.105  ] -33.4871482849
[ 0.3   0.06] -35.4825248718
[ 0.27234375  0.061875  ] -36.5919799805
[ 0.28828125 -0.069375  ] -35.3591804504
[  2.25000000e-01  -1.88737914e-15] -39.0803337097
[ 0.1696875  0.00375  ] -42.5732765198
[ 0.15375  0.135  ] -40.1728591919
[ 0.05109375  0.076875  ] -52.2553672791
[-0.05953125  0.084375  ] -39.9548072815
[ 0.06703125 -0.054375  ] -40.2744522095
[ 0.08871094 -0.00703125] -45.6235466003
[-0.02988281  0.06609375] -42.131904602

dc:  9.31e-02+5.63e-02j	I:  7.26e-01-6.55e-03j	Q: 4.34e-01+6.98e-01j	B: -28.68 G: -32.99, C:0.00
[-62.22344971 -87.8536911  -32.98948669 -57.74613571 -29.52266884
 -80.86745453 -58.30720901]
dc:  9.89e-02+5.71e-02j	I:  7.81e-01-1.54e-01j	Q: 4.04e-01+7.62e-01j	B: -28.57 G: -31.30, C:0.00
[-62.96825027 -84.85345459 -31.30474663 -54.6884079  -29.57398033
 -79.71915436 -57.29652023]
dc:  9.73e-02+5.17e-02j	I:  7.93e-01-8.65e-02j	Q: 4.02e-01+7.50e-01j	B: -28.48 G: -31.74, C:0.00
[-62.57330322 -84.20661163 -31.73701096 -56.48588181 -29.38350487
 -79.10263824 -57.56240845]
dc:  1.06e-01+5.42e-02j	I:  8.46e-01-2.46e-01j	Q: 4.08e-01+7.62e-01j	B: -28.34 G: -30.60, C:0.00
[-63.21781158 -81.61535645 -30.6034832  -53.14982986 -29.42704201
 -79.34053802 -56.84792709]
dc:  1.19e-01+5.38e-02j	I:  9.62e-01-5.07e-01j	Q: 3.77e-01+8.36e-01j	B: -28.40 G: -28.69, C:0.08
[-63.99371338 -77.80194855 -28.69405746 -55.75740814 -29.47135735
 -69.62314606 -55.49826813]
dc:  1.03e-01+5.69e-02j	I:  8.76e-01-2.61e-01

KeyboardInterrupt: 

### Длительность и амплитуда считывающих импульсов

In [ ]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*15/16
    
    print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [ ]:
ro_adc_length = 0.9e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
ro_amplitude = 0.8
#ro_adc_length = 0.3e-6
#ro_amplitude = 0.5
#ro_amplitude = 1.0
ro_dac_length = 0.3e-6
adc.set_nums(10000)
#adc.set_nums(1000)
#adc.set_software_nums_multi(1)
set_adc_nop(ro_adc_length)

In [ ]:
#tomography.set_sequence([tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)], awg_channels)
plt.plot(np.mean(np.real(adc.measure()['Voltage']), axis=0))

In [ ]:
plt.pcolormesh(np.imag(adc.measure()['Voltage']));
plt.colorbar()

In [ ]:
adc.stop()

In [ ]:
sa.set_xlim(qubit_params[qubit_id]['Fr']-200e6, qubit_params[qubit_id]['Fr']+200e6)
plt.plot(sa.get_tracedata()[0], sa.get_tracedata()[1])
pna.get_frequency()

In [ ]:
lo_amp.set_status(0)

### Прохождение считывающих испульсов разной амплитуды

In [ ]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0, 1, 51), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

In [ ]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0, 1, 1), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

In [ ]:
lo1.set_status(1)

### Осцилляции Раби

In [ ]:
sa.set_centerfreq(8e9)
sa.set_span(200e6)
sa.set_nop(201)
f, P = sa.get_tracedata()
plt.figure()
plt.plot(f, P)

In [ ]:
lo_amp.set_status(1)

In [15]:
ex_amplitude = 0.2
pause_length = 128e-9
lengths = np.linspace(0, 0.2e-6, 101)
readout_begin = np.max(lengths)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length'), filename='Rabi')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

A Jupyter Widget

Started at:  Jun 21 2018 17:12:46


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


NameError: name 'ro_dac_length' is not defined

In [ ]:
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

### Осцилляции Рамзея

In [ ]:
ex_amplitude = 0.2
#pi2_pulse = 6e-9
pi2_pulse = 9e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 1000e-9, 101)
target_freq_offset = 10e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi2_pulse),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey', 'name': 'qubit {} paramp {}'.format(qubit_id, 'on' if lo_amp.get_status() else 'off')}, measurement_fitted, annotation=annotation)

### Спиновое эхо

In [ ]:
ex_amplitude = 0.2
pi2_pulse = 9e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 500e-9, 101)
target_freq_offset = 10e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi2_pulse),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays*2, set_delay, 'Spin echo full delay'), filename='Spin echo')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Spin echo', 'name': 'qubit {} paramp {}'.format(qubit_id, 'on' if lo_amp.get_status() else 'off')}, measurement_fitted, annotation=annotation)

In [ ]:
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

In [ ]:
lo_amp.set_status(0)

### Затухание свободной индукции (T1)

In [ ]:
ex_amplitude = 0.2
pi_pulse = 18e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 2000e-9, 51)
readout_begin = np.max(delays)+pi_pulse
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-ro_dac_length),
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
#save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay', 'name': 'qubit {} paramp {}'.format(qubit_id, 'on' if lo_amp.get_status() else 'off')}, measurement_fitted, annotation=annotation)

In [ ]:
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

### Rabi amplitude oscillations

In [ ]:
ex_amps = np.linspace(0.75,1.0,21)
pause_length = 32e-9
pi2_length = 16e-9
sigma = 4e-9
num_pulses = 16
readout_begin = np.max((pi2_length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', pi2_length, pg.gauss_hd,  amp, 0, sigma,0)]*num_pulses+\
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    #print(sequence)
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

### gaussian $\pi/2$ pulse amplitudes

In [ ]:
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
amp_x = 0.90
length = 16e-9
pause_length = 0e-9
sigma = 4e-9
amp_x_hd = 0.89
length_hd = 16e-9
sigma_hd = 4e-9
pause_length_hd = 8e-9
alpha_hd = -0.92e-9

### Ramsey with gauss pulses

In [ ]:
delays = np.linspace(0e-9, 2000e-9, 201)
target_freq_offset = 20e6

def set_delay(delay):
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude, 0), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi), 0), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

# Optimizing readout pulse settings

In [ ]:
#ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma), pg.p(None, pause_length)]*2
ex_calib_seq = [pg.p('iq_ex', 18e-9, pg.rect, 0.2), pg.p(None, pause_length)]
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc_sz.save_last_samples = True
#adc_sz.dump_measured_samples = True
adc.set_software_nums_multi(1)
adc_sz.repeat_samples = 3

def target_min():
    F = 1-adc_sz.measure()['Calibrated ROC AUC']
    print ('fidelity: ', 1-F)
    return F

class ro_pulse_generator:
    def __init__(self, adc_sz, ro_amplitude, ro_dac_length):
        self.ro_amplitude = ro_amplitude
        self.ro_dac_length = ro_dac_length
        self.adc_sz = adc_sz
    def set_ro_freq(self, ro_freq):
        iq_ro.set_uncal_frequency(ro_freq)
        self.make_ro_seq()
    def set_paramp_pump_freq(self, pump_freq):
        lo_amp.set_frequency(pump_freq)
    def set_paramp_pump_power(self, pump_power):
        lo_amp.set_power(pump_power)
    def set_paramp_bias_current(self, paramp_bias):
        current_src2.set_current(paramp_bias)
    def make_ro_seq(self):
        adc_sz.ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', self.ro_dac_length, pg.rect, self.ro_amplitude)]
    def set_ro_length(self, ro_dac_length):
        self.ro_dac_length = ro_dac_length
        self.make_ro_seq()
        adc_sz.measurement_name = 'ro_dac_length {0}ns ro_dac_amiltude {1:4.2f}'.format(int(self.ro_dac_length*1e9), self.ro_amplitude)
    def set_ro_amplitude(self, ro_amplitude):
        self.ro_amplitude = ro_amplitude
        self.make_ro_seq()
        adc_sz.measurement_name = 'ro_dac_length {0}ns ro_dac_amiltude {1:4.2f}'.format(int(self.ro_dac_length*1e9), self.ro_amplitude)

ro_dac_lengths = np.linspace(0.05e-6,1.0e-6,20)
ro_dac_amplitudes = np.linspace(0.05,1.0,20)

ro_pulse_generator_inst = ro_pulse_generator(adc_sz, ro_amplitude, ro_dac_length)
#solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 3e-8), 
#                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.9), maxfun=30)
#ro_dac_length = solution[0]
#ro_dac_amplitude = solution[1]
#

In [ ]:
sweep.sweep(adc_sz, (ro_dac_lengths, ro_pulse_generator_inst.set_ro_length, 'Readout pulse length'), 
                    (ro_dac_amplitudes, ro_pulse_generator_inst.set_ro_amplitude, 'Readout pulse amplitude'), filename='Readout fidelity with paramp')

## Optimizing readout pulse length, amplitude, frequency, paramteric amplifier pump and bias

In [ ]:
p0 = [700e-9, 0.5, iq_ro.get_frequency(), lo_amp.get_frequency(), lo_amp.get_power(), current_src2.get_current()]
dp = [50e-9, 0.05, 0.5e6, 3e6, 0.06, 3e-5]

initial_simplex = [p0]+[[p0[i]+(dp[i] if i == j else 0) for i in range(len(dp))] for j in range(len(dp))]

solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 700e-9), 
                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.5),
                                             (ro_pulse_generator_inst.set_ro_freq, iq_ro.get_frequency()),
                                             (ro_pulse_generator_inst.set_paramp_pump_freq, lo_amp.get_frequency()),
                                             (ro_pulse_generator_inst.set_paramp_pump_power, lo_amp.get_power()),
                                             (ro_pulse_generator_inst.set_paramp_bias_current, current_src2.get_current()),
                                             initial_simplex=initial_simplex,
                                             maxfun=50)

### |0> & |1> readout calibration

In [ ]:
lo_amp.get_status()

In [ ]:
imp.reload(tomography_new)
imp.reload(awg_iq)
imp.reload(save_pkl)

In [ ]:
ro_dac_length = 600e-9
ro_dac_amplitude = 0.517
ro_delay_length = 10e-9
ro_delay_amplitude = 1.
#ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]*2
ex_calib_seq = [pg.p('iq_ex', 59e-9, pg.rect, 0.5), pg.p(None, pause_length)]
#ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma)]*2
#ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
ro_delay_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_delay_length, pg.rect, ro_delay_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg, ro_delay_seq = ro_delay_seq)
adc_sz.save_last_samples = True
adc_sz.repeat_samples = 1
#adc.set_software_nums_multi(2)
print(adc_sz.measure_delay('iq_ro'))
print(adc_sz.measure())
#sz_filter = adc_sz.calibrate()
#print('ROC AUC:', adc_sz.calib_roc_auc, 'Fidelity:', adc_sz.calib_fidelity)
#print('ROC AUC binary:', adc_sz.calib_roc_auc_binary, 'Fidelity binary:', adc_sz.calib_fidelity_binary) 

In [ ]:
adc_sz.dump_samples('ro_dac_length {0}ns ro_dac_amiltude {1:4.2f}'.format(int(ro_dac_length*1e9), ro_dac_amplitude))

In [ ]:
adc.stop()

In [ ]:
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[0,:,:]), vmin=15, vmax=25); plt.colorbar()
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[1,:,:]), vmin=15, vmax=25); plt.colorbar()

In [ ]:
plt.hist(adc_sz.calib_pred.T, bins=20)

In [ ]:
#plt.scatter(np.real(adc_sz.calib_usl_pred[0,:]), np.imag(adc_sz.calib_usl_pred[0,:]), color='r', s=1)
#plt.scatter(np.real(adc_sz.calib_usl_pred[1,:]), np.imag(adc_sz.calib_usl_pred[1,:]), color='b', s=1)
#plt.figure()
#plt.hist(adc_sz.calib_pred.T, bins=20)
plt.figure()
#plt.hist(np.real(adc_sz.calib_usl_pred.T), bins=20)
plt.plot(np.real(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.imag(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.real(adc_sz.calib_feature))
plt.plot(np.imag(adc_sz.calib_feature))

### Tomography

In [ ]:
observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Xo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq, 
                 'operator': observables['X']},
            'Yo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['Y']},
            '-Xo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-X']},
            '-Yo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-Y']},
            'Zo': {'pulses':ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'x':{'operator':observables['X']},
                      'y':{'operator':observables['Y']},
                      'z':{'operator':observables['Z']}}
tomo = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [ ]:
proj_seq = {'Z': {'pulses':ro_seq, 'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq)

In [ ]:
## tomography of |0> state
adc.set_software_nums_multi(20)
tomo.set_prepare_seq([])
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*2)
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*10)
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]*10)
tomo.measure()

In [ ]:
## tomography of |0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of |0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, 1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of -|0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of -|0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

# Randomized benchmarking

In [ ]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomo)

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}
bench.interleavers = {'X': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]])},
                      'Y': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]])},
                      '-X':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]])},
                      '-Y':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])},
                      'I':{'pulses':[], 'unitary':np.asarray([[1, 0], [0,1]])}}
bench.sequence_length = 10
bench.random_sequence_num = 6
bench.prepare_random_interleaving_sequences()
#bench.generate_interleaver_sequence_from_names()

In [ ]:
seq_lengths = np.arange(101)
sweep.sweep(bench, (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number'))

In [ ]:
bench.target_gate = bench.interleavers['X']['pulses']
bench.target_gate_unitary = bench.interleavers['X']['unitary']
bench.target_gate_name = 'X (benchmarking)'
tfunc = lambda : np.mean(bench.measure()['Euclidean distance'])
class hd_pulse_generator:
    def __init__(self, pulse_setter, pg, channel):
        self.amp_x = -amp_x_hd
        self.amp_y = amp_x_hd
        self.sigma = sigma_hd
        self.length = length_hd
        self.pause_length = pause_length_hd
        self.alpha = alpha_hd
        
        self.pg = pg
        self.channel = channel
        self.pulse = self.pulse_gen()
        self.pulse_setter = pulse_setter
    def set_real_amplitude(self,x):
        self.amp_x = x
        self.pulse_setter(self.pulse_gen())
    def set_imag_amplitude(self,y):
        self.amp_y = y
        self.pulse_setter(self.pulse_gen())
    def set_sigma(self,s):
        self.sigma = s
        self.pulse_setter(self.pulse_gen())
    def set_length(self,l):
        self.length = l
        self.pulse_setter(self.pulse_gen())
    def set_pause_length(self, p):
        self.pause_length = p
        self.pulse_setter(self.pulse_gen())
    def set_alpha(self, a):
        self.alpha = a
        self.pulse_setter(self.pulse_gen())
    def pulse_gen(self):
        return [pg.p(self.channel, self.length, self.pg.gauss_hd, self.amp_x+1j*self.amp_y, 0, self.sigma, self.alpha), pg.p(None, self.pause_length)]
    
xg = hd_pulse_generator(bench.set_target_pulse, pg, 'iq_ex')

In [ ]:
sol, score = sweep.optimize(tfunc,  (xg.set_real_amplitude, xg.amp_x), 
                                    (xg.set_imag_amplitude, xg.amp_y),
                                    (xg.set_alpha, xg.alpha))

In [ ]:
amps = np.linspace(0.925, 0.975, 21)
alphas = np.linspace(-2e-9, 2e-9, 11)
sweep.sweep(bench, (amps, xg.set_real_amplitude, 'Amplitude'), (alphas, xg.set_alpha, 'HD DRAG coeff'), filename='Randomized interleaved benchmarging X-rotation')

In [ ]:
data = save_pkl.load_pkl('Euclidean distance Randomized interleaved benchmarging X-rotation', 'D:\\qtlab\\ReiData\\data\\2017-07-02\\23-03-34')
plt.figure()
plt.pcolormesh(data[1]['Euclidean distance'][1][1], data[1]['Euclidean distance'][1][0], data[1]['Euclidean distance'][2][:,:,5])
plt.colorbar()

In [ ]:
data[1]

In [ ]:
data[1]['Euclidean distance']

In [ ]:
meas = bench.measure()

In [ ]:
meas['Xo']

In [ ]:
import plotting
imp.reload(plotting)
plotting.plot_measurement(bench.reference_benchmark_result)

# Optimizing Drag

In [ ]:
num_interleved_pulses = 2
alphas = np.linspace(-1e-8, 1e-8, 201)
def set_drag (alpha):
    ramsey_half = [pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                 pg.p(None, pause_length_hd)]
    interleaved_identity = [pg.p('iq_ex', length_hd, pg.gauss_hd, -amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd),
                           pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd)]
    tomo.set_prepare_seq(ramsey_half+interleaved_identity*num_interleved_pulses+ramsey_half)
    
measurement = sweep.sweep(tomo, (alphas, set_drag, 'delta_freq'), filename='DRAG', output=False)

### Randomized Benchmarking

In [ ]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.linspace(0, 100, 11)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [ ]:
rho_m

In [ ]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [ ]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
m

In [ ]:
fidelities

In [ ]:
psis

In [ ]:
rho_m

In [ ]:
rho_t

In [ ]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

In [ ]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

In [ ]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

In [ ]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

In [ ]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

### Optimizing phase between x and y pulses

In [ ]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

In [ ]:
phases[7]

In [ ]:
plt.plot(np.asarray(results).T)

In [ ]:
adc.stop()

In [ ]:
plt.close()

# load calibration data

In [ ]:
import pickle
f0 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-46\\Voltage ro calibration.pkl', 'rb')
f1 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-14\\Voltage ro calibration.pkl', 'rb')
cal0 = pickle.load(f0)
cal1 = pickle.load(f1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt4
cal0_nomean = cal0[1]['Voltage'][2][0,:,:]-np.mean(cal0[1]['Voltage'][2][0,:,:])
cal1_nomean = cal0[1]['Voltage'][2][1,:,:]-np.mean(cal0[1]['Voltage'][2][1,:,:])

cal0_mean = np.mean(cal0_nomean, axis=0)
cal1_mean = np.mean(cal1_nomean, axis=0)

In [ ]:
cal_mean = (cal0_mean+cal1_mean)/2.
cal_diff = (cal1_mean-cal0_mean)

plt.figure('Samples')
plt.plot(np.mean(cal0_nomean, axis=0), label='|0>')
plt.plot(np.mean(cal1_nomean, axis=0), label='|1>')
plt.legend()

plt.figure('Offset & feature')
plt.plot(cal_mean, label='mean')
plt.plot(cal_diff, label='diff')
plt.legend()

In [ ]:
plt.figure('Fourier domain')
plt.plot(np.abs(np.fft.fft(cal0_mean)), label='|0>')
plt.plot(np.abs(np.fft.fft(cal1_mean)), label='|1>') 
plt.plot(np.abs(np.fft.fft(cal_mean)), label='mean')
plt.plot(np.abs(np.fft.fft(cal_diff)), label='diff')
plt.legend()

feature = np.conj(cal_diff/np.sum(np.abs(cal_diff**2)))

In [ ]:
coeffs0 = np.dot(cal0_nomean-cal_mean, feature)
coeffs1 = np.dot(cal1_nomean-cal_mean, feature)
predictions = np.asarray([np.real(coeffs0), np.real(coeffs1)])

In [ ]:
hist_all, bins = np.histogram(predictions, bins='auto')
proba_points = (bins[1:]+bins[:-1])/2.
hists = []
for y in range(2):
    hists.append(np.histogram(predictions[y,:], bins=bins)[0])

hists = np.asarray(hists, dtype=float)
probabilities = hists/hist_all
naive_probabilities = np.asarray([proba_points<0, proba_points>0], dtype=float)
probabilities[np.isnan(probabilities)] = naive_probabilities[np.isnan(probabilities)]
predictor = lambda x: np.interp(x, proba_points, probabilities[1,:], left=0., right=1.)
calib_proba_points = proba_points
calib_proba = probabilities[1,:]
calib_hists = hists

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
roc = roc_curve([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))
roc_auc = roc_auc_score([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))

In [ ]:
plt.figure('ROC')
plt.plot(roc[0], roc[1])
plt.xlabel('True Positive Rate')
plt.ylabel('True Negative Rate')
roc_auc

In [ ]:
plt.figure('probability curve')
plt.plot(calib_proba_points, calib_proba)

In [ ]:
plt.figure('Readout hists')
plt.bar(proba_points-0.25, hists[0], width=0.05, label='|0>')
plt.bar(proba_points+0.25, hists[1], width=0.05, label='|1>')
plt.legend()

In [ ]:
plt.close()

In [ ]:
cal0_nomean.shape

In [ ]:
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal0_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal1_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')

In [ ]:
plt.plot(np.mean(cal0_nomean, axis=0))
plt.plot(np.mean(cal1_nomean, axis=0))

### extract samples from scan from their directories and put into a new directory

In [ ]:
import os
import glob
new_root = 'D:\\qtlab\\ReiData\\data\\2018-03-13\\Paramp-on-samples\\'
path = 'D:\\qtlab\\ReiData\\data\\2018-03-13\\Paramp-on-samples\\day2\\*\\*.pkl'
pickles = glob.glob(path)
for pickle in pickles:
    pickle_name = pickle[(pickle.rfind('\\')+1):]
    amplitude = 0.05
    pickle_renamed = pickle_name.replace('ro_dac_amiltude 1.00', 'ro_dac_amplitude {0:.2f}'.format(amplitude))
    i = 0
    while(os.path.exists(new_root+pickle_renamed)):
        amplitude += 0.05
        pickle_renamed = pickle_name.replace('ro_dac_amiltude 1.00', 'ro_dac_amplitude {0:.2f}'.format(amplitude))
        i += 1 
        if i> 100:
            raise(Exception('infinite loop '+pickle_renamed))
    #print (pickle_name)
    os.rename(pickle, new_root+pickle_renamed)

### zip samples

In [ ]:
import zipfile
sources = 'D:\\qtlab\\ReiData\\data\\2018-03-13\\Paramp-on-samples\\*.pkl'
pickles = glob.glob(sources)
for pickle in pickles:
    #pickle_name = pickle[(pickle.rfind('\\')+1):]
    print (pickle)
    zout = zipfile.ZipFile(pickle+'.zip', "w", zipfile.ZIP_DEFLATED)
    zout.write(pickle)
    zout.close()